# How to use NCBI Variation Services to compare frequencies for two populations for variations from a VCF file

## Overview

This tutorial will show how to get frequencies in African and Asian populations given VCF lines. There are 4 steps.

1. Set up table headers metadata
2. Convert VCF data lines to GRCh38 in [SPDI format](https://www.ncbi.nlm.nih.gov/variation/notation/)
3. Retrieve all frequencies from the `overlapping_frequency_records` endpoint
4. Display



## Preparation

We will need several packages:
* `pyvcf` - parse the input VCF
* `requests` - access the Variation APIs
* `ratelimit` - respect the volume limits on Variation APIs
* `tqdm` - progress bar because some operations are slow
* `tabulate` - final display


In [ ]:
%pip install -q pyvcf requests ratelimit tabulate tqdm

In [ ]:
from ratelimit import limits, sleep_and_retry
from tqdm import tqdm
import requests
import vcf

# Python library imports
from collections import namedtuple
from io import StringIO
from operator import attrgetter, itemgetter
import itertools

### flatten() utility function

This should be part of the language

In [ ]:
def flatten(iterable):
    '''Return a list that flattens one level of nesting in nested lists'''
    return list(itertools.chain.from_iterable(iterable))

### `get()` utility function

We'll be accessing the API a lot. The `get()` function below deals with some of the boilerplate.

In [ ]:
VAR_API_URL = "https://api.ncbi.nlm.nih.gov/variation/v0/"

@sleep_and_retry
@limits(calls=1, period=1)  # Limit request rate to 1 RPS
def get(endpoint, **params):
    """
    Return the result of a Variation Services endpoint GET queery as a Python data structure.
    Obeys API rate limits. Raises an exception on request failure.
    """
    # Use the production SPDI service
    api_url = "https://www.ncbi.nlm.nih.gov/variation/v0/" \
        if endpoint.startswith('spdi') else VAR_API_URL
    reply = requests.get(VAR_API_URL + endpoint, params=params)
    reply.raise_for_status()
    return reply.json()

### `Spdi` tuple

SPDIs as named tuples are easier to work with than as dictionaries. The field names were chosen so that the way NCBI services return SPDI in JSON is easily convertible. If you have a dictionary `spdi_dict` from a service, you can convert it using `Spdi(**spdi_dict)`. As an added safeguard, Python will complain if there are any extra or missing fields.

In [ ]:
Spdi = namedtuple('Spdi', 'seq_id position deleted_sequence inserted_sequence')

## Table headers / population ids

### Populations we are interested in

NCBI uses Biosample and Bioproject IDs to precisely identify the populations and studies to which it refers. The constants below give those IDs easier to remember names. You can find them at the `metadata/frequency` endpoint.

In [ ]:
# dbGaP's Allele Frequency Aggregator (ALFA) project
BIOPROJECT_ID = "PRJNA507278"

AAA_BIOSAMPLE_ID = 'SAMN10492703' # All African Ancestry
ASN_BIOSAMPLE_ID = 'SAMN10492704' # Asian

### Table to convert biosample ids to human readable population names

`POPULATION_NAMES` will hold a dictionary that converts biosample ids to human-readable names derived from the `metadata/frequency` endpoint. This will be used  later to display table headers.

In [ ]:
POPULATION_NAMES = {}

def add_populations(root_pop):
    for pop in root_pop:
        if 'subs' in pop:
            add_populations(pop['subs'])
        POPULATION_NAMES[pop['biosample_id']] = pop['name']

for bioproject in get("metadata/frequency"):
    add_populations(bioproject['populations'])

## Input

Below are our 20 variants of interest.

In [ ]:
INPUT_VCF = """##fileformat=VCFv4.0
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO
1	82154	.	A	G	.	.	.
1	91472	.	C	A,G,T	.	.	.
1	570178	.	G	A	.	.	.
1	724702	.	G	A,C,T	.	.	.
1	726481	.	T	G	.	.	.
1	752566	.	G	A,T	.	.	.
1	752721	.	A	C,G	.	.	.
1	753541	.	G	A	.	.	.
1	754182	.	A	G	.	.	.
1	755775	.	A	C,G,T	.	.	.
1	758770	.	C	A	.	.	.
1	760300	.	A	G	.	.	.
1	760912	.	C	T	.	.	.
1	762320	.	C	T	.	.	.
1	768448	.	G	A	.	.	.
1	776546	.	A	G	.	.	.
1	779322	.	A	G	.	.	.
1	785989	.	T	C	.	.	.
1	798959	.	G	A	.	.	.
1	800007	.	T	C	.	.	.
"""

INPUT_VCF_ASSEMBLY = 'GCF_000001405.25'

Parsing a small VCF file is a one-liner in Python.

In [ ]:
VCF_LINES = list(vcf.Reader(StringIO(INPUT_VCF)))

## Conversion to SPDI and remapping

The `vcf/.../contextuals` endpoint converts the first four VCF fields to a "contextual" SPDI. This means that the input variant will be expanded as required to eliminate ambiguity. The resulting SPDI will then be passed to the `canonical_representative` endpoint to remap it to the latest RefSeq assembly.

In [ ]:
VCF_LINES_IN_SPDI_FORMAT = []

def remap(spdi):
    remap_url = 'spdi/{}:{}:{}:{}/canonical_representative'.format(*spdi)
    return Spdi(**get(remap_url)['data'])

for record in tqdm(VCF_LINES):
    alts = ','.join(map(str, record.ALT))
    query_url = f'vcf/{record.CHROM}/{record.POS}/{record.REF}/{alts}/contextuals'
    spdis_for_alts = [Spdi(**spdi_dict) for spdi_dict in 
                      get(query_url, assembly=INPUT_VCF_ASSEMBLY)['data']['spdis']]

    # If the VCF is not on the GRCh38 assembly, remap each SPDI to GRCh38.
    if INPUT_VCF_ASSEMBLY != 'GCF_000001405.38':
        spdis_for_alts = [remap(spdi) for spdi in spdis_for_alts]

    VCF_LINES_IN_SPDI_FORMAT.append(spdis_for_alts)

## Retrieving frequency data

For every line of the input VCF file, we query frequency data for the range that spans all corresponding SPDIs. This reduces the number of `http` requests needed.

In [ ]:
FREQUENCIES_FOR_VCF_LINES = []

for spdis_for_alts in tqdm(VCF_LINES_IN_SPDI_FORMAT):
    seq_id = spdis_for_alts[0].seq_id
    min_pos = min(map(attrgetter('position'), spdis_for_alts))
    max_pos_plus_one = max(map(lambda s: s.position + len(s.deleted_sequence), spdis_for_alts))

    frequency_records = get(
        'interval/{}:{}:{}/overlapping_frequency_records'.format(
            seq_id,
            min_pos,
            max_pos_plus_one - min_pos))['results']

    FREQUENCIES_FOR_VCF_LINES.append((seq_id, frequency_records))

## Reporting

There may be more frequencies returned than there were original records since the service returns all variants that overlap the region. So, we need to report only those frequencies that correspond to alleles in the original file.

First, we will prepare the frequency records for later look-up. The keys in the `ALLELE_FREQUENCIES` dictionary are SPDI tuples of the alleles.

In [ ]:
ALLELE_FREQUENCIES = {}

def compute_frequencies(allele_counts):
    total = float(sum(allele_counts.values()))
    if total:
        return {allele : count / total for allele, count in allele_counts.items()}
    else:
        return {allele: 0.0 for allele in allele_counts.keys()}

for seq_id, frequency_records in FREQUENCIES_FOR_VCF_LINES:
    for interval, interval_data in frequency_records.items():
        length, position = map(int, interval.split('@'))
        # Get counts for the dbGaP Allele Frequency Aggregation, ALFA, project.
        allele_counts = interval_data['counts'][BIOPROJECT_ID]['allele_counts']
        
        aaa_frequencies = compute_frequencies(allele_counts[AAA_BIOSAMPLE_ID])
        asn_frequencies = compute_frequencies(allele_counts[ASN_BIOSAMPLE_ID])

        for allele in asn_frequencies.keys():
            ALLELE_FREQUENCIES[Spdi(seq_id, position, interval_data['ref'], allele)] = \
                (aaa_frequencies[allele],
                 asn_frequencies[allele])

Now we use the `ALLELE_FREQUENCIES` dictionary to attach frequency information to the SPDIs generated from the input VCF lines.

In [ ]:
TABLE_LINES = []

def ref_spdi(spdi):
    """Spdi representing the reference allele"""
    return Spdi(*attrgetter('seq_id','position',
                            'deleted_sequence','deleted_sequence')(spdi))

for vcf_line_number, spdis_for_alts in enumerate(VCF_LINES_IN_SPDI_FORMAT,
                                                 start=1):
    unique_alleles = set(flatten((s, ref_spdi(s)) for s in spdis_for_alts))

    for allele in unique_alleles:
        TABLE_LINES.append((vcf_line_number,
                            allele.seq_id, 
                            allele.position,
                            len(allele.deleted_sequence),
                            '\tRef' if allele.deleted_sequence == allele.inserted_sequence else ' ',
                            allele.inserted_sequence,) +
                           ALLELE_FREQUENCIES.get(allele, ('N/A', 'N/A')))

Now all that remains is to display the table constructed above.

In [ ]:
from tabulate import tabulate
from IPython.display import display, HTML

display(HTML(tabulate(sorted(TABLE_LINES),
                      tablefmt='html',
                      headers=['VCF Line',
                               'Chromosome',
                               'Position',
                               'Len',
                               'Ref',
                               'Allele',
                               POPULATION_NAMES[AAA_BIOSAMPLE_ID],
                               POPULATION_NAMES[ASN_BIOSAMPLE_ID]])))

VCF Line,Chromosome,Position,Len,Ref,Allele,All African Ancestry,Asian
1,NC_000001.11,82153,1,Ref,A,1,1
1,NC_000001.11,82153,1,,G,0,0
2,NC_000001.11,91471,1,Ref,C,0.94157,1
2,NC_000001.11,91471,1,,A,0,0
2,NC_000001.11,91471,1,,G,0,0
2,NC_000001.11,91471,1,,T,0.0584304,0
3,NC_000001.11,634797,1,Ref,G,0.696336,0
3,NC_000001.11,634797,1,,A,0.303664,1
4,NC_000001.11,789321,1,Ref,G,1,1
4,NC_000001.11,789321,1,,A,0,0
